In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D, ReLU, Flatten

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
samples_per_class = 5000
no_of_classes = 3

In [4]:
noise_df = pd.read_csv("/content/gdrive/MyDrive/GW_SOP/Data/Final_Merged_Noise_Reduced_No_Abs.csv", header=None)
noise = noise_df.values.astype(float)


data_BBH_df = pd.read_csv("/content/gdrive/MyDrive/GW_SOP/Data/Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv", header=None)
data_BBH = data_BBH_df.values.astype(float)


data_BNS_df = pd.read_csv("/content/gdrive/MyDrive/GW_SOP/Data/Final_BNS_Merged_Noise_Signal_Reduced_No_ABS.csv", header=None)
data_BNS = data_BNS_df.values.astype(float)

In [5]:
X = np.concatenate((noise, data_BBH, data_BNS), axis=0)

print(len(noise_df.index))
print(X)
print(X.shape)

5000
[[-1.64907037e-19 -4.29360853e-20  3.12423747e-20 ... -2.04472673e-19
  -1.81142159e-19 -1.51305791e-20]
 [ 9.92276697e-20 -1.80029547e-19 -1.23077317e-19 ...  6.22533943e-20
  -9.09674752e-20  2.12523082e-19]
 [-6.44325773e-21  5.41100166e-20 -6.99219049e-20 ... -1.41370244e-20
  -3.97475053e-20 -7.86069176e-20]
 ...
 [-3.23880911e-21 -7.34737312e-21  1.11265679e-20 ... -3.75220314e-20
   3.05405540e-20 -7.12104955e-20]
 [-2.26903379e-19  4.35581982e-20 -6.56942361e-20 ... -6.74037998e-20
   5.64279348e-20 -1.39437967e-19]
 [-9.73289950e-20  8.23111862e-20 -1.72899438e-20 ...  6.34110814e-20
   3.22702001e-20  1.48113480e-20]]
(15000, 16384)


In [6]:
# Alternate way of creating y for the dataset
 
y = [int(i/samples_per_class) for i in range(samples_per_class*no_of_classes)]
y = tf.keras.utils.to_categorical(y)
print(y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X)

[[-1.13101151 -0.26478354  0.26752609 ... -1.98185536 -1.79452258
  -0.13759256]
 [ 0.76810283 -1.25778699 -0.85894784 ...  0.6207804  -0.90246737
   2.18404605]
 [ 0.00833455  0.43814651 -0.47093394 ... -0.1246152  -0.3957726
  -0.78493168]
 ...
 [ 0.03137436 -0.00700528  0.12068849 ... -0.35279933  0.2995537
  -0.7095021 ]
 [-1.57676187  0.36171696 -0.44007359 ... -0.64437697  0.55564523
  -1.40529381]
 [-0.64512902  0.64241489 -0.08674103 ...  0.63207678  0.31666427
   0.1677587 ]]


In [8]:
X = np.expand_dims(X, axis=-1)
print(X.shape)
print(X)

(15000, 16384, 1)
[[[-1.13101151]
  [-0.26478354]
  [ 0.26752609]
  ...
  [-1.98185536]
  [-1.79452258]
  [-0.13759256]]

 [[ 0.76810283]
  [-1.25778699]
  [-0.85894784]
  ...
  [ 0.6207804 ]
  [-0.90246737]
  [ 2.18404605]]

 [[ 0.00833455]
  [ 0.43814651]
  [-0.47093394]
  ...
  [-0.1246152 ]
  [-0.3957726 ]
  [-0.78493168]]

 ...

 [[ 0.03137436]
  [-0.00700528]
  [ 0.12068849]
  ...
  [-0.35279933]
  [ 0.2995537 ]
  [-0.7095021 ]]

 [[-1.57676187]
  [ 0.36171696]
  [-0.44007359]
  ...
  [-0.64437697]
  [ 0.55564523]
  [-1.40529381]]

 [[-0.64512902]
  [ 0.64241489]
  [-0.08674103]
  ...
  [ 0.63207678]
  [ 0.31666427]
  [ 0.1677587 ]]]


In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.01) #, random_state=1

In [10]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(14850, 16384, 1) (150, 16384, 1) (14850, 3) (150, 3)


In [11]:
model = Sequential()

model.add(Conv1D(64, 32, input_shape = (16384,1)))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Conv1D(128, 64))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Conv1D(256, 64))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Conv1D(512, 128))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Flatten())
model.add(Dense(13824))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))                         # dropout rate not givem
model.add(Dense(3, activation=tf.nn.softmax))

# learning rate not given
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 16353, 64)         2112      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4088, 64)         0         
 )                                                               
                                                                 
 re_lu (ReLU)                (None, 4088, 64)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 4025, 128)         524416    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 1006, 128)        0         
 1D)                                                             
                                                                 
 re_lu_1 (ReLU)              (None, 1006, 128)         0

In [13]:
model.fit(X_train, y_train, batch_size=128, epochs=10) #validation_data=(X_val, y_val)

Epoch 1/10
117/117 [==============================] - 261s 2s/step - loss: 14.1224 - accuracy: 0.3302
Epoch 2/10
117/117 [==============================] - 236s 2s/step - loss: 1.0993 - accuracy: 0.3308
Epoch 3/10
117/117 [==============================] - 235s 2s/step - loss: 1.0993 - accuracy: 0.3267
Epoch 4/10
117/117 [==============================] - 235s 2s/step - loss: 1.0995 - accuracy: 0.3317
Epoch 5/10
117/117 [==============================] - 236s 2s/step - loss: 1.0988 - accuracy: 0.3403
Epoch 6/10
117/117 [==============================] - 236s 2s/step - loss: 1.0989 - accuracy: 0.3324
Epoch 7/10
117/117 [==============================] - 236s 2s/step - loss: 1.0988 - accuracy: 0.3354
Epoch 8/10
117/117 [==============================] - 236s 2s/step - loss: 1.0994 - accuracy: 0.3238
Epoch 9/10
117/117 [==============================] - 236s 2s/step - loss: 1.0988 - accuracy: 0.3373
Epoch 10/10
117/117 [==============================] - 236s 2s/step - loss: 1.0990 - accur